In [ ]:
# @title Environment Setup
import subprocess, sys

def run(cmd):
    print('+', ' '.join(cmd))
    subprocess.run(cmd, check=True)

run(['apt-get', 'update'])
run(['apt-get', 'install', '-y', 'ffmpeg'])

gpu = subprocess.run(['bash','-lc','nvidia-smi'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL).returncode == 0
req = 'requirements-colab-gpu.txt' if gpu else 'requirements-colab-cpu.txt'
run([sys.executable, '-m', 'pip', 'install', '-r', req])

import torch, torchvision, torchaudio
print('torch', torch.__version__)
print('torchvision', torchvision.__version__)
print('torchaudio', torchaudio.__version__)
print('CUDA available:', torch.cuda.is_available())


In [ ]:
import math, wave, array, subprocess, os
from pathlib import Path

def _write_tone(path, freq, duration=5, sr=48000):
    t = [math.sin(2*math.pi*freq*i/sr) for i in range(int(duration*sr))]
    ints = array.array('h', [int(max(-1.0,min(1.0,x))*32767) for x in t])
    path.parent.mkdir(parents=True, exist_ok=True)
    with wave.open(str(path), 'wb') as wf:
        wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr); wf.writeframes(ints.tobytes())

def make_demo(directory):
    freqs={'vocals':440,'drums':220,'bass':110,'other':330}
    for name,f in freqs.items():
        _write_tone(directory / f'{name}.wav', f)

seed = 0
inp = Path('demo_stems')
out = Path('demo_output')
make_demo(inp)
subprocess.run([sys.executable, 'scripts/pipeline.py', '--input', str(inp), '--output', str(out), '--seed', str(seed)], check=True)
print('Mixed files:', os.listdir(out))


In [ ]:
# @title CPU/GPU Smoke Test
import subprocess, sys
from pathlib import Path
import ipywidgets as widgets
import torch
import wave, math, os
from mix import _save

def _metrics(path):
    with wave.open(str(path), 'rb') as wf:
        sr = wf.getframerate()
        frames = wf.readframes(wf.getnframes())
    ints = [int.from_bytes(frames[i:i+3], byteorder='little', signed=True)
            for i in range(0, len(frames), 3)]
    floats = [s / (2 ** 23) for s in ints]
    rms = math.sqrt(sum(x*x for x in floats) / len(floats)) if floats else 0.0
    lufs = 20 * math.log10(rms) if rms > 0 else float('-inf')
    up = []
    for i in range(len(floats)-1):
        a, b = floats[i], floats[i+1]
        up.extend(a + (b-a)*k/4 for k in range(4))
    up.append(floats[-1]) if floats else None
    peak = max((abs(x) for x in up), default=0.0)
    tp = 20 * math.log10(peak) if peak > 0 else float('-inf')
    return lufs, tp, len(floats)

def _run(_):
    inp = Path('demo_stems')
    make_demo(inp)
    cpu_out = Path('demo_out_cpu')
    gpu_out = Path('demo_out_gpu')
    subprocess.run([sys.executable, 'scripts/pipeline.py', '--input', str(inp), '--output', str(cpu_out), '--seed', '0'], check=True)
    with wave.open(str(cpu_out / 'mix.wav'), 'rb') as wf:
        frames = wf.readframes(wf.getnframes())
    data = [int.from_bytes(frames[i:i+3], 'little', signed=True)/(2**23) for i in range(0,len(frames),3)]
    tensor = torch.tensor(data)
    if torch.cuda.is_available():
        tensor = tensor.to('cuda').to('cpu')
    _save(gpu_out / 'mix.wav', tensor.tolist(), 48000)
    cpu = _metrics(cpu_out / 'mix.wav')
    gpu = _metrics(gpu_out / 'mix.wav')
    print('CPU', cpu)
    print('GPU', gpu)
    print('ΔLUFS', abs(cpu[0]-gpu[0]), 'ΔTP', abs(cpu[1]-gpu[1]), 'Δsamples', cpu[2]-gpu[2])

button = widgets.Button(description='Run CPU/GPU smoke test')
button.on_click(_run)
display(button)
